https://www.smartplay.lcsd.gov.hk/facilities/select/court?venueId=226&fatId=509&venueName=%E4%BD%95%E6%96%87%E7%94%B0%E9%AB%94%E8%82%B2%E9%A4%A8&sessionIndex=0,1&dateIndex=0&playDate=2025-03-17&district=KC,KT,SSP,WTS,YTM&typeCode=TABT&sportCode=BAGM&frmFilterType=&isFree=false

In [5]:
import requests
from datetime import datetime
import pandas as pd

### Extract Available Timeslot Response From Smartplay.gov

In [6]:
def fetch_available_venue_timeslot(district_codes, facility_code, play_date):
    
    base_url = "https://www.smartplay.lcsd.gov.hk/rest/facility-catalog/api/v1/publ/facilities"

    params = {
        "distCode": ",".join(district_codes),
        "faCode": facility_code,
        "playDate": play_date
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
        'Referer': 'https://www.smartplay.lcsd.gov.hk/facilities/search-result',
        'channel': 'INTERNET'
    }

    response = requests.get(base_url, params=params, headers=headers)
    return response.json()

district_codes=['KC', 'KT', 'SSP', 'WTS', 'YTM']
facility_code='TABT'
play_date='2025-03-18'

available_venue_timeslot = fetch_available_venue_timeslot(district_codes, facility_code, play_date)
available_venue_timeslot

{'code': '0',
 'message': 'success',
 'data': {'morning': {'distList': [{'distCode': 'KC',
     'sessionCount': 32,
     'distName': 'Kowloon City',
     'venueList': [{'venueName': 'Fat Kwong Street Sports Centre',
       'venueId': 223,
       'venueImageUrl': 'https://lcsd-smartplay.oss-cn-hongkong.aliyuncs.com/pic/default_facility.jpeg',
       'sessionCount': 5,
       'fatList': [{'fatName': 'Table Tennis Table (A/C)(Urban)',
         'enFatName': 'Table Tennis Table (A/C)(Urban)',
         'fatId': 509,
         'sessionCount': 5,
         'faCodeName': 'Table Tennis',
         'faCode': 'TABT',
         'faGroupCode': 'BAGM',
         'fvrId': 125005431,
         'sessionList': [{'ssnStartTime': '07:00:00',
           'ssnEndTime': '08:00:00',
           'ssnStartDate': '2025-03-18',
           'available': True,
           'peakHour': False,
           'sfadFlag': False,
           'sessionCount': 2},
          {'ssnStartTime': '08:00:00',
           'ssnEndTime': '09:00:00',


### Data Pre-processing of Available Venue Timeslot

In [7]:
def process_available_venue_timeslot(available_venue_timeslot):
    data = []

    for period in [i for i in available_venue_timeslot["data"].keys() if i != "venueCountList"]:
        print(available_venue_timeslot["data"][period])

        for district in available_venue_timeslot["data"][period]["distList"]:
            # print(district["distCode"])

            for venue in district["venueList"]:
                # print(" ", venue["venueName"])
                
                for facility in venue["fatList"]:
                    # print("  ", facility["fatName"])

                    for session in facility["sessionList"]:
                        # print("   ", session["ssnStartTime"], session["ssnEndTime"], session["available"])

                        session_info = {"date": session["ssnStartDate"],
                                        "period": period,
                                        "district_code": district["distCode"],
                                        "district_name": district["distName"],
                                        "venue_id": venue["venueId"],
                                        "venue_name": venue["venueName"],
                                        "facility_id": facility["fatId"],
                                        "facility_name": facility["fatName"],
                                        "session_start_time": session["ssnStartTime"],
                                        "session_end_time": session["ssnEndTime"],
                                        "session_available": session["available"]}
                        
                        data.append(session_info)
                        # print(session_info)
    return data

data = process_available_venue_timeslot(available_venue_timeslot)
pd.DataFrame(data)

{'distList': [{'distCode': 'KC', 'sessionCount': 32, 'distName': 'Kowloon City', 'venueList': [{'venueName': 'Fat Kwong Street Sports Centre', 'venueId': 223, 'venueImageUrl': 'https://lcsd-smartplay.oss-cn-hongkong.aliyuncs.com/pic/default_facility.jpeg', 'sessionCount': 5, 'fatList': [{'fatName': 'Table Tennis Table (A/C)(Urban)', 'enFatName': 'Table Tennis Table (A/C)(Urban)', 'fatId': 509, 'sessionCount': 5, 'faCodeName': 'Table Tennis', 'faCode': 'TABT', 'faGroupCode': 'BAGM', 'fvrId': 125005431, 'sessionList': [{'ssnStartTime': '07:00:00', 'ssnEndTime': '08:00:00', 'ssnStartDate': '2025-03-18', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}, {'ssnStartTime': '08:00:00', 'ssnEndTime': '09:00:00', 'ssnStartDate': '2025-03-18', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}, {'ssnStartTime': '09:00:00', 'ssnEndTime': '10:00:00', 'ssnStartDate': '2025-03-18', 'available': False, 'peakHour': False, 'sfadFlag': False, 'sessionCo

,date,period,district_code,district_name,venue_id,venue_name,facility_id,facility_name,session_start_time,session_end_time,session_available
0,2025-03-18,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,509,Table Tennis Table (A/C)(Urban),07:00:00,08:00:00,True
1,2025-03-18,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,509,Table Tennis Table (A/C)(Urban),08:00:00,09:00:00,True
2,2025-03-18,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,509,Table Tennis Table (A/C)(Urban),09:00:00,10:00:00,False
3,2025-03-18,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,509,Table Tennis Table (A/C)(Urban),10:00:00,11:00:00,False
4,2025-03-18,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,509,Table Tennis Table (A/C)(Urban),11:00:00,12:00:00,True
...,...,...,...,...,...,...,...,...,...,...,...
995,2025-03-18,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,509,Table Tennis Table (A/C)(Urban),18:00:00,19:00:00,True
996,2025-03-18,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,509,Table Tennis Table (A/C)(Urban),19:00:00,20:00:00,False
997,2025-03-18,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,509,Table Tennis Table (A/C)(Urban),20:00:00,21:00:00,False
998,2025-03-18,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,509,Table Tennis Table (A/C)(Urban),21:00:00,22:00:00,False


### Extract District Info Code

In [9]:
def fetch_district_info():
    """Fetches and structures district hierarchy data from the LCSD API."""
    
    # Main response structure
    api_response = {
        "fetch_date": datetime.today().strftime("%Y-%m-%d"),
        "districts": []
    }

    request_headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://www.smartplay.lcsd.gov.hk/",
        "Accept": "application/json"
    }

    districts_endpoint = "https://www.smartplay.lcsd.gov.hk/rest/param/api/v1/publ/districts/searching-criteria?pgm=N"

    server_response = requests.get(districts_endpoint, headers=request_headers)
    response_data = server_response.json().get("data", 0)

    if response_data:

        for region_group in response_data:
            region_dict = {
                "region_id": region_group["id"],
                "region_code": region_group["code"].strip(),
                "region_name": region_group["name"].strip(),
                "sc_name": region_group["scName"].strip(),
                "tc_name": region_group["tcName"].strip(),
                "subdistricts": []
            }

            for subdistrict in region_group["children"]:
                subdistrict_dict = {
                    "subdistrict_id": subdistrict["id"],
                    "subdistrict_code": subdistrict["code"].strip(),
                    "subdistrict_name": subdistrict["name"].strip(),
                    "en_name": subdistrict["enName"].strip(),
                    "sc_name": subdistrict["scName"].strip(),
                    "tc_name": subdistrict["tcName"].strip()
                }
                region_dict["subdistricts"].append(subdistrict_dict)

            api_response["districts"].append(region_dict)

        return api_response
    
    return {"status": "error", "code": server_response.status_code}

district_info = fetch_district_info()
district_info

{'fetch_date': '2025-03-14',
 'districts': [{'region_id': 202313440,
   'region_code': 'HK',
   'region_name': 'Hong Kong',
   'sc_name': '香港',
   'tc_name': '香港',
   'subdistricts': [{'subdistrict_id': 202311860,
     'subdistrict_code': 'CW',
     'subdistrict_name': 'Central & Western',
     'en_name': 'Central & Western',
     'sc_name': '中西区',
     'tc_name': '中西區'},
    {'subdistrict_id': 202311861,
     'subdistrict_code': 'EN',
     'subdistrict_name': 'Eastern',
     'en_name': 'Eastern',
     'sc_name': '东区',
     'tc_name': '東區'},
    {'subdistrict_id': 202311869,
     'subdistrict_code': 'SN',
     'subdistrict_name': 'Southern',
     'en_name': 'Southern',
     'sc_name': '南区',
     'tc_name': '南區'},
    {'subdistrict_id': 202311873,
     'subdistrict_code': 'WCH',
     'subdistrict_name': 'Wan Chai',
     'en_name': 'Wan Chai',
     'sc_name': '湾仔区',
     'tc_name': '灣仔區'}]},
  {'region_id': 202313441,
   'region_code': 'KLN',
   'region_name': 'Kowloon',
   'sc_name': '九

In [10]:
def fetch_facility_data():
    url = "https://www.smartplay.lcsd.gov.hk/rest/facility-catalog/api/v1/publ/facility-params/facilities?ballotFlag=false"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0",
        "Referer": "https://www.smartplay.lcsd.gov.hk/facilities/home",
        "Accept": "application/json"
    }

    response_json = requests.get(url, headers=headers).json()

    response_data = response_json.get("data", 0)

    if response_data:
        return response_data


def process_facility_data(raw_data):

    processed = []
    
    for category in raw_data:
        print(category)
        # Extract category-level information
        category_info = {
            'Category Code': category['code'],
            'Category Name (EN)': category['enName'],
            'Category Name (TC)': category['tcName'],
            'Category Name (SC)': category['scName']
        }
        
        # Process child facilities
        for facility in category.get('children', []):
            facility_data = {
                'Facility Code': facility['code'],
                'Facility Name (EN)': facility['enName'],
                'Facility Name (TC)': facility['tcName'],
                'Facility Name (SC)': facility['scName'],
                'Non-Fee Flag': facility['nonFee']
            }
            
            # Combine with category info
            processed.append({**category_info, **facility_data})
    
    return pd.DataFrame(processed)


# Use the previously defined fetch function
raw_data = fetch_facility_data()

df = process_facility_data(raw_data)
df[:20]

{'code': 'ARCH', 'name': 'Archery', 'enName': 'Archery', 'tcName': '箭藝', 'scName': '箭艺', 'disp': True, 'seq': 1, 'nonFee': False, 'icon': 'archery', 'params': {'icon': 'archery'}, 'children': [{'code': 'ARH', 'groupCode': 'ARCH', 'name': 'Archery', 'enName': 'Archery', 'tcName': '箭藝', 'scName': '箭艺', 'disp': True, 'seq': 1, 'nonFee': False}]}
{'code': 'BAGM', 'name': 'Ball Games', 'enName': 'Ball Games', 'tcName': '球類運動', 'scName': '球类运动', 'disp': True, 'seq': 2, 'nonFee': True, 'icon': 'ballgames', 'params': {'icon': 'ballgames'}, 'children': [{'code': 'AMPL', 'groupCode': 'BAGM', 'name': 'American Pool', 'enName': 'American Pool', 'tcName': '美式桌球', 'scName': '美式桌球', 'disp': True, 'seq': 1, 'nonFee': False}, {'code': 'NFBVOLC', 'groupCode': 'BAGM', 'name': 'Beach Volleyball (Non-fee)', 'enName': 'Beach Volleyball (Non-fee)', 'tcName': '沙灘排球 (不收費)', 'scName': '沙滩排球 (不收费)', 'disp': True, 'seq': 1, 'nonFee': True}, {'code': 'FHSP', 'groupCode': 'BAGM', 'name': 'FHSP', 'enName': 'FHSP', '

,Category Code,Category Name (EN),Category Name (TC),Category Name (SC),Facility Code,Facility Name (EN),Facility Name (TC),Facility Name (SC),Non-Fee Flag
0,ARCH,Archery,箭藝,箭艺,ARH,Archery,箭藝,箭艺,False
1,BAGM,Ball Games,球類運動,球类运动,AMPL,American Pool,美式桌球,美式桌球,False
2,BAGM,Ball Games,球類運動,球类运动,NFBVOLC,Beach Volleyball (Non-fee),沙灘排球 (不收費),沙滩排球 (不收费),True
3,BAGM,Ball Games,球類運動,球类运动,FHSP,FHSP,五人硬地足球,五人硬地足球,False
4,BAGM,Ball Games,球類運動,球类运动,SPG,Sportsground,運動場,运动场,False
5,BAGM,Ball Games,球類運動,球类运动,BADC,Badminton,羽毛球,羽毛球,False
6,BAGM,Ball Games,球類運動,球类运动,BVOLC,BVOLC,沙灘排球場,沙滩排球场,False
7,BAGM,Ball Games,球類運動,球类运动,NFRHOC,Roller Hockey (Non-fee),滾軸曲棍球 (不收費),滚轴曲棍球 (不收费),True
8,BAGM,Ball Games,球類運動,球类运动,SHSP,SHSP,七人硬地足球,七人硬地足球,False
9,BAGM,Ball Games,球類運動,球类运动,NFARH,Archery (Non-fee),箭藝 (不收費),箭艺 (不收费),True
